# V8 대비 프롬프트 수정 사항

-----------

+ 문장 생성 프롬프트
  + buykorea 설명에서 문장 요약 시, 웹 검색 지시문 제거
  
    + Use publicly available product descriptions from websites to help. -> 제거

-----------

+ 추출 프롬프트
  + 키워드, 구 추출 기준
    + product's main feature -> product's strong point and purpose

  + 이름도 추출해 중복 방지 (용량, 제품 PID 제거)
  + 키워드, 구 추출 시 one-shot 예시 추가
  + 키워드 개수 제한 지시문 기존보다 강력한 어조로 수정

-----------

+ 임베딩 유형 merge
  + 유형 필터, 통 문장
  + 유형 필터, 구절+통 문장
  + 유형 필터, 키워드

In [2]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

from google.colab import output

output.clear()

In [3]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [4]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [5]:
bl_data_sampled=pd.read_csv('bl_sample_data_842952.csv')

display(bl_data_sampled)

,HSCD,SUPLY_CO_NAME,PRDT_DC_VAL
0,842952,DOOSAN BOBCAT EMEA S.R.O.,"4 PCS LOADER T450 S N AUVP17400, AUVP17399, AU..."
1,842952,ZHAOHUAN,1PCS EAR WRAP CRAWLER HOOK EARRINGS<br/>
2,842952,MAEDA SEISAKUSHO CO LTD,MAEDA CRAWLER CRANE CE MAR<br/>
3,842952,CATERPILLAR AMERICAS CV,UNIT NEW CATERPILLAR 308 MINI EXCAVATOR HS COD...
4,842952,KOBELCO CONSTRUCTION MACHINERY CO,PS04013178 SK55SRX 6E LH04006019 ED160BR 7 LH0...
...,...,...,...
2280,842952,VOLVO CONSTRUCTION EQUIPMENT,VOLVO EC480DL CRAWLER EXCAVATOR (S.NO:275971) ...
2281,842952,VOLVO CONSTRUCTION EQUIPMENT,VOLVO EC480DL CRAWLER EXCAVATOR (S.NO:275970 P...
2282,842952,VOLVO CONSTRUCTION EQUIPMENT,VOLVO EC480DL CRAWLER EXCAVATOR (S.NO:275982) ...
2283,842952,VOLVO CONSTRUCTION EQUIPMENT,VOLVO EC480DL CRAWLER EXCAVATOR (S.NO:275972) ...


In [7]:
bk_data_sampled=pd.read_excel('BK_842952.xlsx')

display(bk_data_sampled.head())

,pid,product_name,company_name,product_description,url
0,3705754,Electric Mini Excavator,"HR E&I CO.,LTD",• Product name: iX35e\n• Brand name: HR E&I\n•...,https://buykorea.org/ec/prd/selectGoodsDetail....
1,3578239,Spare Parts of Excavator and Forklift Parts,SESAN INDUSTRY CO.,1) Parts of Hyundai forklifts and excavators.\...,https://buykorea.org/ec/prd/selectGoodsDetail....
2,3707438,Hydraulic Breaker V-14OS (Engcon Oil Quick Ded...,"JSC CO., LTD.",Hydraulic breaker is a construction equipment ...,https://buykorea.org/ec/prd/selectGoodsDetail....
3,3704794,HY Breaker,k-breaker,• Product name (Product name) TF-400\n• Brand ...,https://buykorea.org/ec/prd/selectGoodsDetail....


# Langsmith , LLM API KEY 설정

In [8]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [9]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [10]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=8,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-10-d541afd0af05>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_openai = InMemoryRateLimiter(


# 사용 모델 설정

In [11]:
# chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini")

# OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 업무 배경 설명 (제품명, 상품 설명에 대한 데이터임을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 제품명과 상품 정보에 기반한 제품 설명 생성
+ 지시 내용별 예시 제공 (one-shot)

In [12]:
keyword_num=5

class DescriptionSummary(BaseModel):
    name : str = Field(description="product name")
    type : str = Field(description="product type")
    keyword : list[str] = Field(description=f"product keywords, which focus on product's strong point and purpose")
    phrase : list[str] = Field(description=f"product phrases, which focus on product's strong point and purpose")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [13]:
summary_prompt_buykorea = PromptTemplate.from_template(
    """You are given data about product's description(PRDT_DESC) and product name(PRDT_NAME).
    All products fall under HS Code 842952, which is a subcategory of 8429.
    The description for 8429 is as follows:
    84.29 Self-propelled bulldozers, angledozers, graders, levellers, scrapers, mechanical shovels, excavators, shovel loaders, tamping machines and road rollers.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DESC using both PRDT_NAME and PRDT_DESC.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Example :
    PRDT_DESC : 'FIORESE Pure Hydra Water Cream contains Centella Asiatica and Hyaluronic Acid, effectively delivering hydration and providing instant soothing benefits.
    With Niacinamide and Adenosine, it is a dual-functional cosmetic for brightening and wrinkle care. This cream helps create healthy, smooth, and radiant skin.
    The lightweight, hydrating formula absorbs quickly, delivering moisture deep into the skin with a fresh, non-sticky finish. Suitable for all skin types, especially sensitive, dull, and uneven skin.

    For Sensitive Skin: Gentle, irritation-free formula, suitable even for sensitive skin.

    FIORESE Brand: Fiorese is dedicated to clean beauty, using ingredients sourced from nature. 100% cruelty-free.'

    Output Example : 'The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients.'

    PRODUCT DESCRIPTION(PRDT_DESC) : {prdt_desc}
    PRODUCT NAME(PRDT_NAME) : {prdt_name}
    """
)


summary_chain_buykorea = summary_prompt_buykorea | chat_openai

In [14]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = summary_chain_buykorea.invoke({"prdt_desc": row['product_description'], "prdt_name": row['product_name']})
  print(response_buykorea.content)
  buykorea_preprocessed_dict_openai[row['product_name']]=[response_buykorea.content]

The iX35e Electric Mini Excavator, developed by HR E&I, is a versatile and eco-friendly machine designed for various applications such as agriculture, horticulture, gardening, construction, civil engineering, and demolition. With dimensions of 2,180mm x 1,555mm x 2,540mm and a weight of 3,950kg, it is compact yet powerful. 

This mini excavator features a variable track width, allowing for adaptability in different environments, and offers a power mode for enhanced performance. It can be operated remotely from up to 200 meters away, ensuring safety during unmanned excavation. The iX35e is designed with zero emissions, making it an environmentally friendly choice. 

Additionally, it comes with a mobile charger and an option for a fast charger, providing convenience for users. The excavator boasts a normal work runtime of 8 hours and a continuous work runtime of 6 to 7 hours, depending on the task, making it efficient for various projects. Overall, the iX35e is a reliable and sustainable

In [17]:
extract_prompt_buykorea = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, ensuring that the number of keywords is exactly {{keyword_num}}. Do not exceed or fall short of this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT EXAMPLE : ''' The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients'''

    OUTPUT EXAMPLE : '''
    {{'name' : 'FIORESE Pure Hydra Water Cream',
     'type' : 'Cream',
     'keyword' :  ['Lightweight', 'Hydrating', 'Brightening', 'Wrinkle care', 'Sensitive skin'],
     'phrase' : ['Contains Centella Asiatica and Hyaluronic Acid for deep moisture and soothing benefits',
                 'Dual-functional cosmetic for brightening and wrinkle care with Niacinamide and Adenosine',
                 'Quick absorption with a fresh, non-sticky finish, suitable for all skin types']}}
    '''

    INPUT : {input}
    OUTPUT:{{'name' : '<product name>', 'type': '<product type>', 'keyword': [<keyword1>. <keyword2>, ...], 'phrase' : [<phrase1>. <phrase2>, ...]}}
    """
)

extract_prompt_buykorea= extract_prompt_buykorea.partial(keyword_num=keyword_num)

extract_chain_buykorea = extract_prompt_buykorea | chat_openai | parser

In [18]:
for key, value in buykorea_preprocessed_dict_openai.items():
  response_buykorea = extract_chain_buykorea.invoke({"input": value})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.name)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.type)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.keyword)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.phrase)

name='iX35e Electric Mini Excavator' type='Excavator' keyword=['Eco-friendly', 'Versatile', 'Remote operation', 'Zero emissions', 'Compact'] phrase=['Designed for various applications such as agriculture, horticulture, gardening, construction, civil engineering, and demolition', 'Features a variable track width for adaptability in different environments', 'Operated remotely from up to 200 meters away for safety during unmanned excavation', 'Includes a mobile charger and an option for a fast charger for user convenience', 'Reliable and sustainable solution for modern excavation needs']
name='Spare Parts of Excavator and Forklift Parts' type='Spare Parts' keyword=['Hydraulic', 'Compatible', 'Essential', 'Reliable', 'Comprehensive'] phrase=['Wide range of components designed for various brands such as Hyundai, Doosan, Volvo, and Daewoo', 'Includes hydraulic breaker assemblies, seal kits for hydraulic cylinders, pumps, and motors', 'Encompasses attachments for forklifts and other construct

In [19]:
buykorea_preprocessed_dict_openai

{'Electric Mini Excavator': ['The iX35e Electric Mini Excavator, developed by HR E&I, is a versatile and eco-friendly machine designed for various applications such as agriculture, horticulture, gardening, construction, civil engineering, and demolition. With dimensions of 2,180mm x 1,555mm x 2,540mm and a weight of 3,950kg, it is compact yet powerful. \n\nThis mini excavator features a variable track width, allowing for adaptability in different environments, and offers a power mode for enhanced performance. It can be operated remotely from up to 200 meters away, ensuring safety during unmanned excavation. The iX35e is designed with zero emissions, making it an environmentally friendly choice. \n\nAdditionally, it comes with a mobile charger and an option for a fast charger, providing convenience for users. The excavator boasts a normal work runtime of 8 hours and a continuous work runtime of 6 to 7 hours, depending on the task, making it efficient for various projects. Overall, the i

In [20]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['description_summary','name', 'type', 'keyword', 'phrase'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,description_summary,name,type,keyword,phrase
0,Electric Mini Excavator,"The iX35e Electric Mini Excavator, developed b...",iX35e Electric Mini Excavator,Excavator,"[Eco-friendly, Versatile, Remote operation, Ze...",[Designed for various applications such as agr...
1,Spare Parts of Excavator and Forklift Parts,The Spare Parts of Excavator and Forklift Part...,Spare Parts of Excavator and Forklift Parts,Spare Parts,"[Hydraulic, Compatible, Essential, Reliable, C...",[Wide range of components designed for various...
2,Hydraulic Breaker V-14OS (Engcon Oil Quick Ded...,The Hydraulic Breaker V-14OS (Engcon Oil Quick...,Hydraulic Breaker V-14OS,Construction Attachment,"[Demolition, Hydraulic Power, Versatile, Effic...",[Engineered for demolition tasks and breaking ...
3,HY Breaker,The HY Breaker by k-breaker is a robust side a...,HY Breaker,Side Attachment,"[Robust, Durable, Constant performance, Heat r...",[Designed for shredding applications with cons...


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ 업무 배경 설명 (수입자, 공급자, 상품 설명이 존재함을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 공급자와 상품 이름 정보에 기반한 제품 설명 생성
+ 인터넷 검색으로 추가 정보 검색 지시
+ 의미가 확실치 않은 경우 생성 제한
+ 지시 내용별 예시 제공 (one-shot)

In [21]:
generate_prompt_bldata = PromptTemplate.from_template(
    """You are given data where IMP_CO_NAME imports products (PRDT_DC_VAL) from SUPLY_CO_NAME.
    All products fall under HS Code 842952, which is a subcategory of 8429.
    The description for 8429 is as follows:
    84.29 Self-propelled bulldozers, angledozers, graders, levellers, scrapers, mechanical shovels, excavators, shovel loaders, tamping machines and road rollers.

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DC_VAL using both SUPLY_CO_NAME and PRDT_DC_VAL.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Input Example:
    PRDT_DC_VAL: COSMELAN 2 (FACE CREAM)
    SUPLY_CO_NAME: MESOESTETIC, S.L.
    Output Example: 'This product is called COSMELAN 2, a face cream designed to reduce pigmentation and improve skin tone.
    It is often used for treating melasma and other skin discolorations by inhibiting melanin production.'


    PRODUCT DESCRIPTION(PRDT_DC_VAL) : {prdt_desc}
    SUPPLEYER NAME(SUPLY_CO_NAME) : {suply_name}
    """
)

generate_chain_bldata = generate_prompt_bldata | chat_openai

In [ ]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = generate_chain_bldata.invoke({"prdt_desc": row['PRDT_DC_VAL'], "suply_name":row['SUPLY_CO_NAME']})
  if index%100==0:
    print(f"\nProduct Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata.content, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=[response_bldata.content]

1it [00:04,  4.48s/it]


Product Name : 4 PCS LOADER T450 S N AUVP17400, AUVP17399, AUVP17401, AUVP17402 2 PCS EXCAVATORS E20 S N B3BL18241, B3BL18255 HS-CODE 84295191, 84295210<br/>5 PCS LOADER T450 S N AUVP17405, AUVP17407, AUVP17406, AUVP17404, AUVP17403 4 PCS EXCAVATOR E10 S N B4K912459, B4 

This product description includes various heavy machinery items imported from DOOSAN BOBCAT EMEA S.R.O. The items listed are:

- 4 units of the Loader T450, identified by serial numbers AUVP17400, AUVP17399, AUVP17401, and AUVP17402. Loaders like the T450 are versatile machines used for moving materials, grading, and other construction tasks.

- 2 units of the Excavator E20, with serial numbers B3BL18241 and B3BL18255. Excavators are essential for digging, lifting, and demolition work, making them crucial in construction and earthmoving projects.

- Additionally, there are 5 more units of the Loader T450, with serial numbers AUVP17405, AUVP17407, AUVP17406, AUVP17404, and AUVP17403, and 4 units of the Excavator E10, 

101it [04:02,  1.78s/it]


Product Name : KOMATSU CONST. MACHINERY NEW HYDRAULIC EXCAV ATOR PC170LC 11 INVOICE NO.CV04215 HS CODE 842952 DISASSEMBLEDPARTS NEW<br/> 

This product is a new hydraulic excavator model PC170LC-11 from Komatsu, a leading manufacturer in construction machinery. The excavator is designed for various earthmoving tasks, featuring advanced hydraulic systems for efficient operation. It is categorized under HS Code 842952, which includes self-propelled machinery like excavators. The mention of "disassembled parts" suggests that this may refer to components of the excavator that are sold separately or require assembly upon delivery. 



201it [07:41,  1.87s/it]

...

2101it [1:22:47,  2.38s/it]


Product Name : DOOSAN NEW CRAWLER EXCAVATOR MODEL NO.DX225LCA-2 WITH STANDARD ACCESSORIES(M/C S.NO.CEBDW-001048) 

This product is a Doosan New Crawler Excavator, specifically the model DX225LCA-2, which comes with standard accessories. Crawler excavators are heavy construction equipment used for digging, lifting, and moving materials on construction sites. The DX225LCA-2 is known for its efficiency and versatility, making it suitable for various applications such as excavation, grading, and demolition. It is manufactured by Doosan Infracore Co. Ltd., a reputable company in the construction machinery industry. 



2201it [1:27:03,  2.26s/it]


Product Name : EXCAVATOR MODEL. E27 (SERIAL NO. B4R811222) (ENGINE NO. 1KL1775) (ITEM NO. M5547*4473113) 

This product is an Excavator Model E27, which is a type of construction equipment used for digging, lifting, and moving materials. The serial number B4R811222 and engine number 1KL1775 indicate specific details about this particular machine, while the item number M5547*4473113 helps in identifying it within the supplier's inventory. Manufactured by Doosan Bobcat EMEA S.R.O., this excavator is designed for efficiency and versatility in various construction and excavation tasks. 



2285it [1:30:28,  2.38s/it]


In [ ]:
joblib.dump(bldata_preprocessed_dict_openai, 'bldata_preprocessed_dict_openai.pkl')

In [33]:
bldata_preprocessed_dict_openai_ext=joblib.load('bldata_preprocessed_dict_openai.pkl')

In [24]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "product name", "title": "Name", "type": "string"}, "type": {"description": "product type", "title": "Type", "type": "string"}, "keyword": {"description": "product keywords, which focus on product\'s strong point and purpose", "items": {"type": "string"}, "title": "Keyword", "type": "array"}, "phrase": {"description": "product phrases, which focus on product\'s strong point and purpose", "items": {"type": "string"}, "title": "Phrase", "type": "array"}}, "required": ["name", "type", "keyword", "ph

In [34]:
extract_prompt_bldata = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, ensuring that the number of keywords is exactly {{keyword_num}}. Do not exceed or fall short of this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT EXAMPLE : ''' The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients'''

    OUTPUT EXAMPLE : '''
    {{"name" : "FIORESE Pure Hydra Water Cream",
     "type" : "Cream",
     "keyword" :  ["Lightweight", "Hydrating", "Brightening", "Wrinkle care", "Sensitive skin"],
     "phrase" : ["Contains Centella Asiatica and Hyaluronic Acid for deep moisture and soothing benefits",
                 "Dual-functional cosmetic for brightening and wrinkle care with Niacinamide and Adenosine",
                 "Quick absorption with a fresh, non-sticky finish, suitable for all skin types"]}}
    '''

    INPUT : {input}
    OUTPUT:{{'name' : '<product name>', 'type': '<product type>', 'keyword': [<keyword1>. <keyword2>, ...], 'phrase' : [<phrase1>. <phrase2>, ...]}}
    """
)

extract_prompt_bldata= extract_prompt_bldata.partial(keyword_num=keyword_num)

extract_chain_bldata = extract_prompt_bldata | chat_openai | parser

In [35]:
for i, (key, value) in tqdm(enumerate(bldata_preprocessed_dict_openai_ext.items())):
  try:
    response_bldata = extract_chain_bldata.invoke({"input": value[0]})
    if i%100==0:
      print(response_bldata)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.name)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.type)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.keyword)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.phrase)
  except:
    print(f"key : {key}")
    print(f"extract error: {response_bldata}")

1it [00:02,  2.02s/it]

name='DOOSAN BOBCAT Machinery' type='Heavy Machinery' keyword=['Versatile', 'Essential', 'Construction', 'Earthmoving', 'Self-propelled'] phrase=['Loaders like the T450 are used for moving materials, grading, and other construction tasks', 'Excavators are crucial for digging, lifting, and demolition work', 'Self-propelled machinery used in construction and earthmoving']


101it [03:00,  1.71s/it]

name='CASE Hydraulic Excavators' type='Excavator' keyword=['Powerful', 'Versatile', 'Heavy construction', 'Earthmoving', 'Reliable'] phrase=['Designed for heavy construction and earthmoving tasks with powerful performance', 'Versatile machines suitable for various construction applications', 'High-quality construction equipment known for reliability and efficiency']


197it [05:53,  2.17s/it]

...

2101it [1:12:38,  1.82s/it]

name='SANY SY35U Excavator' type='Excavator' keyword=['Compact', 'Versatile', 'Efficient', 'Maneuverable', 'Advanced technology'] phrase=['Designed for various excavation tasks with powerful engine', 'Known for efficiency and maneuverability in tight spaces', 'Enhances performance and operator comfort for construction and landscaping']


2201it [1:16:31,  2.12s/it]

name='SY380C9I2KW' type='Excavator' keyword=['Heavy-duty', 'Robust', 'Advanced hydraulic', 'Efficient', 'Reliable'] phrase=['Designed for heavy-duty construction and excavation tasks', 'Features a robust structure and advanced hydraulic systems', "Part of SANY's SY series known for efficiency and reliability in earth-moving applications"]


2245it [1:18:01,  2.09s/it]


In [36]:
joblib.dump(bldata_preprocessed_dict_openai_ext, 'bldata_preprocessed_dict_openai_ext.pkl')

['bldata_preprocessed_dict_openai_ext.pkl']

In [37]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai_ext, orient='index', columns=['product_description','name', 'type', 'keyword', 'phrase'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai.head())

,product_name,product_description,name,type,keyword,phrase
0,"4 PCS LOADER T450 S N AUVP17400, AUVP17399, AU...",This product description includes various heav...,DOOSAN BOBCAT Machinery,Heavy Machinery,"[Versatile, Essential, Construction, Earthmovi...",[Loaders like the T450 are used for moving mat...
1,1PCS EAR WRAP CRAWLER HOOK EARRINGS<br/>,"This product is called ""1PCS Ear Wrap Crawler ...",1PCS Ear Wrap Crawler Hook Earrings,Earrings,"[Stylish, Unique design, Elegant, Eye-catching...",[Designed to wrap around the ear for a distinc...
2,MAEDA CRAWLER CRANE CE MAR<br/>,This product is called the MAEDA CRAWLER CRANE...,MAEDA CRAWLER CRANE CE MAR,Crawler Crane,"[Versatile, Powerful, Tracked undercarriage, S...",[Ideal for lifting heavy loads in challenging ...
3,UNIT NEW CATERPILLAR 308 MINI EXCAVATOR HS COD...,This product is a new Caterpillar 308 Mini Exc...,Caterpillar 308 Mini Excavator,Excavator,"[Versatile, Efficient, Durable, Reliable, Comp...",[Designed for various construction and excavat...
4,PS04013178 SK55SRX 6E LH04006019 ED160BR 7 LH0...,This product description includes various mode...,KOBELCO Hydraulic Excavators,Excavator,"[Heavy construction, Digging, Lifting, Versati...",[Articulated arm and bucket for a wide range o...


In [38]:
bldata_preprocessed_openai[bldata_preprocessed_openai['product_name']=='. = 9 DUMPER DT10 CUSTOMER ORDER 4502306242-D0900 4502306243-D0900 4502306244-D0900 4502306246-D0900 4502623897-D0900 4502623898-D0900 4502623899-D0900 4502623907-D0901 4502623908-D0901 PO 53181955 53181959 53182153 53182155 53257799 53257821 53257822 53257828 53257829 MATERIAL NUMBER 1000325588 IMO 9 UN3166 HS-CODE 87041010<br/>. = 4 CRAWLER EZ26 CUSTOMER ORDER 4502674902-E2001 4502674905-E2001 4502674916-E2001 4502674917-E2001 PO 53272147 53272138 53272139 53272142 MATERIAL NUMBER 1000400964 I']

,product_name,product_description,name,type,keyword,phrase
701,. = 9 DUMPER DT10 CUSTOMER ORDER 4502306242-D0...,The provided product descriptions (PRDT_DC_VAL...,None,None,None,None


In [ ]:
df.fillna('None', inplace=True)

In [ ]:
bldat

# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [ ]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

(2285, 3)

In [ ]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

2245

# 임베딩

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

In [ ]:
bldata_preprocessed_openai['product_description'][0]

'This product description includes various heavy machinery items imported from DOOSAN BOBCAT EMEA S.R.O. The items listed are:\n\n- 4 units of the Loader T450, identified by serial numbers AUVP17400, AUVP17399, AUVP17401, and AUVP17402. Loaders like the T450 are versatile machines used for moving materials, grading, and other construction tasks.\n\n- 2 units of the Excavator E20, with serial numbers B3BL18241 and B3BL18255. Excavators are essential for digging, lifting, and demolition work, making them crucial in construction and earthmoving projects.\n\n- Additionally, there are 5 more units of the Loader T450, with serial numbers AUVP17405, AUVP17407, AUVP17406, AUVP17404, and AUVP17403, and 4 units of the Excavator E10, although the serial numbers for the E10 are not fully listed.\n\nOverall, these products are part of the category of self-propelled machinery used in construction and earthmoving, specifically under HS Code 842952, which includes various types of loaders and excavato

#  1. 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

  1%|▏         | 30/2245 [00:15<19:12,  1.92it/s]


KeyboardInterrupt: 

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())

In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v1.csv', index=False)

# 2. 제품 유형 유사도 필터 후 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_description_embedding']=buykorea_preprocessed_openai['description_summary'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_embedding']=bldata_preprocessed_openai['product_description'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [04:02<00:00, 11.53it/s]


In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_description', 'bldata_product_name',
                                           'bldata_type', 'bldata_description', 'type_cosine_similarity', 'description_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_description': [row['description_summary']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'description_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_description_embedding=row['buykorea_description_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_description_embedding=row_bl['bldata_description_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    description_similarity=cosine_similarity(buykorea_description_embedding, bldata_description_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_type']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'description_cosine_similarity']=description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:05, 540.57it/s]
2801it [00:04, 565.62it/s]
2801it [00:05, 478.43it/s]
2801it [00:05, 508.12it/s]
2801it [00:04, 629.17it/s]
2801it [00:04, 610.57it/s]
2801it [00:05, 509.07it/s]
2801it [00:04, 617.96it/s]
2801it [00:04, 575.47it/s]
2801it [00:05, 516.97it/s]


,buykorea_product_name,buykorea_type,bldata_product_name,bldata_type,cosine_similarity
0,Doldori Premium Gold Essence,mask pack,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),face cream,0.443385
1,Doldori Premium Gold Essence,mask pack,COSMELAN 2 (FACE CREAM),face cream,0.443385
2,Doldori Premium Gold Essence,mask pack,ANTI STRESS FACE MASK 100ML (FACE CREAM),face mask,0.761262
3,Doldori Premium Gold Essence,mask pack,BLOT POWDER,setting powder,0.142408
4,Doldori Premium Gold Essence,mask pack,DERMACOLOR CAMOUFLAGE FLUID,camouflage fluid,0.390623


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'description_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v2.csv', index=False)

,buykorea_product_name,buykorea_type,bldata_product_name,bldata_type,cosine_similarity,test_row
0,Doldori Premium Gold Essence,mask pack,COSMETICS MASK PACK,mask pack,1.000000,Highest
1,Doldori Premium Gold Essence,mask pack,ASSORTD FACE PACK (RETURN TO ORIGIN) (EXPORTED...,face pack,0.785882,Highest
2,Doldori Premium Gold Essence,mask pack,AVOCADO HIGH NUTRITION MASK PACK (BRAND-STAROV...,skincare mask,0.764832,Highest
3,Doldori Premium Gold Essence,mask pack,(LABUTE )REVIVE THE SKIN VITAMIN MASK,skincare mask,0.764832,Highest
4,Doldori Premium Gold Essence,mask pack,HIBISCUS & ROSE GLOW MASK (RETURN TO ORIGIN) (...,skincare mask,0.764832,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,sunscreen,HILARY RHODA HIGHLIGHTER & CONTOUR (NT. WT. 12...,highlighter and contour,0.045261,Lowest
28006,t:aim Barrier Sunblock 40ml,sunscreen,PEPPLUS LIFTING PACK (8 PACKS) ( 40 PCS) ( CDS...,lifting pack,0.041898,Lowest
28007,t:aim Barrier Sunblock 40ml,sunscreen,2.14756 YURAQ (SAMPLE-NCV-R&D PURPOSE) (1 X 25...,sample for research and development,0.033819,Lowest
28008,t:aim Barrier Sunblock 40ml,sunscreen,(N.C.V.) - MIBIOME SUSPENSION 50ML/EN (FOR R &...,Research formulation,0.014456,Lowest


# 3. 제품 유형 유사도와 키워드 5개 유사도 순 나열

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x :x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_keyword_embedding']=buykorea_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_keyword_embedding']=bldata_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [07:33<00:00,  6.17it/s]


In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_keyword', 'bldata_product_name', 'bldata_type', 'bldata_keyword', 'type_cosine_similarity' ,'keyword_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_keyword': [row['keyword_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'keyword_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_keyword_embedding=row['buykorea_keyword_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_keyword_embedding=row_bl['bldata_keyword_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    keyword_similarity=cosine_similarity(buykorea_keyword_embedding, bldata_keyword_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['keyword_sentence']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'keyword_cosine_similarity']=keyword_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:04, 654.86it/s]
2801it [00:05, 525.97it/s]
2801it [00:04, 650.49it/s]
2801it [00:04, 655.87it/s]
2801it [00:05, 531.55it/s]
2801it [00:04, 650.00it/s]
2801it [00:04, 652.54it/s]
2801it [00:05, 526.53it/s]
2801it [00:04, 649.62it/s]
2801it [00:04, 628.05it/s]


,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"face cream,'localized fat', 'cellulite', 'body...",0.654006
1,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",COSMELAN 2 (FACE CREAM),"face cream,'pigmentation', 'skin tone', 'melas...",0.710032
2,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",ANTI STRESS FACE MASK 100ML (FACE CREAM),"face mask,'hydration', 'soothing', 'stressed s...",0.659665
3,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",BLOT POWDER,"setting powder,'shine control', 'oil absorptio...",0.650969
4,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",DERMACOLOR CAMOUFLAGE FLUID,"camouflage fluid,'high coverage', 'skin imperf...",0.454845


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'keyword_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v3.csv', index=False)

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity,test_row
0,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",EB BRIGHTER MOISTURE MAS 100ML/3.4FLOZ - (100....,"moisturizing mask,'brightening', 'moisturizing...",0.923858,Highest
1,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",(LABUTE )REVIVE THE SKIN VITAMIN MASK,"skincare mask,'nourishment', 'rejuvenation', '...",0.910223,Highest
2,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",(N.C.V.) - NEUTROGENA BRIGHT BOOSTERMASK-21G S...,"skincare mask,'brightening', 'hydration', 'rad...",0.900369,Highest
3,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",COSMETICS PRODUCT CODE 34400142 YEHWADAM PURE ...,"skincare kit,'brightening', 'skincare', 'hydra...",0.897600,Highest
4,Doldori Premium Gold Essence,"mask pack,'skincare', 'essence', 'hydration', ...",HIBISCUS & ROSE GLOW MASK (RETURN TO ORIGIN) (...,"skincare mask,'hibiscus', 'rose', 'glow', 'hyd...",0.891152,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF 50+', 'PA+++...",WNW BROW PENCIL(0.2GM) (COSMETIC PRODUCTS) RC ...,"eyebrow pencil,'eyebrow', 'cosmetic', 'define'...",0.324040,Lowest
28006,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF 50+', 'PA+++...",CL L BR FINGER CLIPS - LARGE - BROWN 12PC/BAG,"finger clips,'brown', 'grooming', 'beauty', 's...",0.323467,Lowest
28007,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF 50+', 'PA+++...",COSMETICS MASCARA,"mascara,'mascara', 'enhance', 'eyelashes', 'de...",0.321659,Lowest
28008,t:aim Barrier Sunblock 40ml,"sunscreen,'high-protection', 'SPF 50+', 'PA+++...",COSMETICS 11155 U.BEVER SCALP ENERG SHAMP 258M,"shampoo,'scalp energizing', 'promotes healthy ...",0.270411,Lowest


# 4. 제품 유형 유사도와 전체 문장 + 핵심 구절 n개 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['phrase_sentence']=buykorea_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['phrase_sentence']=bldata_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))

# 제품 유형 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장 + 구절로 구성된 문장을 임베딩

buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_description_phrase_embedding']=buykorea_preprocessed_openai['description_summary']+buykorea_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_phrase_embedding']=bldata_preprocessed_openai['product_description']+bldata_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [07:50<00:00,  5.96it/s]


In [ ]:
import re

openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_phrase_description',
                                           'bldata_product_name', 'bldata_type','bldata_phrase_description',
                                           'type_cosine_similarity', 'phrase_description_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_phrase_description': [row['description_summary']+row['phrase_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'phrase_description_cosine_similarity' : [0]*bldata_sample_len,
                              })

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_phrase_description_embedding=row['buykorea_description_phrase_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):
      bldata_type_embedding=row_bl['bldata_type_embedding']
      bldata_phrase_description_embedding=row_bl['bldata_description_phrase_embedding']

      type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
      phrase_description_similarity=cosine_similarity(buykorea_phrase_description_embedding, bldata_phrase_description_embedding)
      similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
      similarity_df.loc[row_count, 'bldata_type']=row_bl['type']
      similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
      similarity_df.loc[row_count, 'phrase_description_cosine_similarity']=phrase_description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())



2801it [00:00, 5656.22it/s]
2801it [00:00, 6141.98it/s]
2801it [00:00, 6160.93it/s]
2801it [00:00, 14655.14it/s]
2801it [00:00, 16827.97it/s]
2801it [00:00, 6388.54it/s]
2801it [00:00, 10005.21it/s]
2801it [00:00, 10248.40it/s]
2801it [00:00, 10568.09it/s]
2801it [00:00, 7486.45it/s]


,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",None,None,0.000000
1,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",None,None,0.000000
2,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",ANTI STRESS FACE MASK 100ML (FACE CREAM),"'hydration', 'soothing', 'stressed skin', 'red...",0.522552
3,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",None,None,0.000000
4,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",None,None,0.000000


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'],
                                                          group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity','phrase_description_cosine_similarity'], ascending=False))

openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.grozupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v4.csv', index=False)

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity,test_row
0,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",EB BRIGHTER MOISTURE MAS 100ML/3.4FLOZ - (100....,"'brightening', 'moisturizing', 'skin hydration...",0.870156,Highest
1,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",I AM LEMON MASK SHEET - BRIGHTENING ( PRODUCT ...,"'brightening', 'hydration', 'lemon extract', '...",0.858402,Highest
2,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",HIBISCUS & ROSE GLOW MASK (RETURN TO ORIGIN) (...,"'hibiscus', 'rose', 'glow', 'hydration', 'skin...",0.835730,Highest
3,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...","ROSE HIP & MAIZE EXFOLIATING MASK, 250 ML X 30...","'exfoliation', 'radiant complexion', 'nourishi...",0.833835,Highest
4,Doldori Premium Gold Essence,"'skincare', 'essence', 'hydration', 'brighteni...",SU CLARIFYING MASK 150ML (150 ML) (270320030) ...,"'clarifying', 'purifying', 'skincare', 'comple...",0.831755,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,"'high-protection', 'SPF 50+', 'PA++++', 'organ...",None,None,0.000000,Lowest
28006,t:aim Barrier Sunblock 40ml,"'high-protection', 'SPF 50+', 'PA++++', 'organ...",None,None,0.000000,Lowest
28007,t:aim Barrier Sunblock 40ml,"'high-protection', 'SPF 50+', 'PA++++', 'organ...",None,None,0.000000,Lowest
28008,t:aim Barrier Sunblock 40ml,"'high-protection', 'SPF 50+', 'PA++++', 'organ...",None,None,0.000000,Lowest


# 5. 제품 유형 유사도와 핵심 구절 n개 임베딩 + 전체 문장 (순서 변화)

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)


bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [20:23<00:00,  2.29it/s]


In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:04, 572.18it/s]
2801it [00:04, 564.22it/s]
2801it [00:04, 643.42it/s]
2801it [00:05, 549.00it/s]
2801it [00:04, 598.39it/s]
2801it [00:04, 645.52it/s]
2801it [00:05, 531.20it/s]
2801it [00:04, 641.18it/s]
2801it [00:04, 639.29it/s]
2801it [00:05, 509.32it/s]


,buykorea_product_name,buykorea_description,bldata_product_name,bldata_description,cosine_similarity
0,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),This product is called BODYSHOCK LOCAL REDUCER...,0.434211
1,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,COSMELAN 2 (FACE CREAM),"This product is called COSMELAN 2, a face crea...",0.477261
2,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,ANTI STRESS FACE MASK 100ML (FACE CREAM),"This product is called ANTI STRESS FACE MASK, ...",0.586053
3,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,BLOT POWDER,"This product is called BLOT POWDER, a setting ...",0.483652
4,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,DERMACOLOR CAMOUFLAGE FLUID,This product is called DERMACOLOR CAMOUFLAGE F...,0.518189


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v6.csv', index=False)

,buykorea_product_name,buykorea_description,bldata_product_name,bldata_description,cosine_similarity,test_row
0,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,GOLD MASK 500 ML,"This product is called GOLD MASK, a skincare t...",0.700289,Highest
1,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,SPA BLK PEEL FACE MASK 50 V1NG (AVON PLANET SP...,"This product is called SPA BLK PEEL FACE MASK,...",0.693363,Highest
2,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,URBAN COLOR PRO WHITE ADVANCE ANTI WRINKLE & N...,This product is called URBAN COLOR PRO WHITE A...,0.685175,Highest
3,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,I AM SEAWEEDS MASK SHEET - PURIFYING (PRODUCT ...,This product is called I AM SEAWEEDS MASK SHEE...,0.682498,Highest
4,Doldori Premium Gold Essence,The Doldori Premium Gold Essence is an innovat...,COSMETIC PRODUCTS V22 DNA MOISTURIZING MASK 250ML,This product is called COSMETIC PRODUCTS V22 D...,0.681528,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,The t:aim Barrier Sunblock is a high-protectio...,PARSLEY (DRIED LEAVES) (FREE SAMPLE),"This product is called PARSLEY (DRIED LEAVES),...",0.171900,Lowest
28006,t:aim Barrier Sunblock 40ml,The t:aim Barrier Sunblock is a high-protectio...,GLITTER POWDER (FOR COSMETIC RAW MATERIAL),"This product is called GLITTER POWDER, which i...",0.171787,Lowest
28007,t:aim Barrier Sunblock 40ml,The t:aim Barrier Sunblock is a high-protectio...,EYE SHADOW PALLETTE (RC NO.- RC/COS-000867 DT....,"This product is called EYE SHADOW PALETTE, whi...",0.168097,Lowest
28008,t:aim Barrier Sunblock 40ml,The t:aim Barrier Sunblock is a high-protectio...,PARSLEY CURLY (DRIED LEAVES) (FREE SAMPLE),"This product is called PARSLEY CURLY, which re...",0.167452,Lowest
